In [1]:
import pandas as pd
import geopandas as gp
import numpy as np
import os
import requests
import urllib.parse

In [2]:
# initiate file paths
input_file_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/aggregated_results_geom.csv"

In [3]:
def drop_levels(df):
    df.reset_index(inplace = True)
    df.columns = df.columns.droplevel(1)
    return df

In [4]:
all_results = pd.read_csv(input_file_path)

In [5]:
sl_by_group = all_results.copy()

# count screening level exceedances by pah, pcb, df, and each individual metal
sl_by_group['Chemical Group'] = np.where(sl_by_group['Chemical Group']=='RCRA8', sl_by_group['Chemical'], sl_by_group['Chemical Group'])

In [6]:
# count of total exceedances by group
cnt_exceedances = sl_by_group[sl_by_group['SL_exceeded']=='Y']
cnt_exceedances = cnt_exceedances.groupby(by = ['DATE','Sample ID','Chemical Group','Medium','Latitude','Longitude']).agg({'SL_exceeded': ['count']}).reset_index()
cnt_exceedances = drop_levels(cnt_exceedances)

In [7]:
# calculate total screening level count
cnt_sl = sl_by_group[sl_by_group['SL_exceeded'] != 'No Screening Level Identified']
cnt_sl = cnt_sl.groupby(by = ['DATE','Sample ID','Chemical Group','Medium','Latitude','Longitude']).agg({'SL_exceeded': ['count']}).reset_index()
cnt_sl = drop_levels(cnt_sl)

In [8]:
cnt_exceedances_merge = cnt_exceedances.merge(cnt_sl, left_on = ['DATE','Sample ID','Chemical Group','Medium','Latitude','Longitude'], right_on = ['DATE','Sample ID','Chemical Group','Medium','Latitude','Longitude'], how = 'left')

In [9]:
cnt_exceedances_merge.rename(columns = {'SL_exceeded_x':'cnt_sl_exceed', 'SL_exceeded_y':'tot_cnt_sl'}, inplace = True)
cnt_exceedances_merge['pct_exceed'] = cnt_exceedances_merge['cnt_sl_exceed'] /cnt_exceedances_merge['tot_cnt_sl']
cnt_exceedances_merge.drop(columns = {'index_x', 'index_y'}, inplace = True)

In [10]:
# export results based on count of exceedances and for each results
cnt_exceedances_merge.to_csv("/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/count_exceedances.csv", index = False)